In [1]:
# Check if python is 3.9.5
import sys
print(sys.version)
%load_ext autoreload
%autoreload 2

3.9.5 (default, May 18 2021, 12:31:01) 
[Clang 10.0.0 ]


In [2]:
from datetime import date
import numpy as np

## Code questions

In [3]:
from utils import download_to_path

In [4]:
from utils import write_csv_row

None

# Data mining

In [5]:
import database_classes
from naip_dbm import NAIPData
from numpy.random import uniform

In [6]:
from multidb_wrapper import GetMultiDBData

In [7]:
from shapely.geometry import Point

In [8]:
lat = 44.967243
lon = -103.771556

# make 10 random points within a given range
pont = Point(float(lon),float(lat))
points = [Point(float(lon+uniform(-.3,.3)),float(lat+uniform(-.3,.3))) for i in range(5)]

In [9]:
# naip = NAIPData(directory="banasna", features=["rgb", "ir"])

In [10]:
# naip.run(points)

In [11]:
# GetMultiDBData(NAIPData(features=["rgb", "ir"]))

In [16]:
a = NAIPData(features=["rgb", "ir"])

The default online source `{self.base_url}` will be used instead.


In [17]:
isinstance(type(a).__base__, database_classes.SpatialData)

False

In [23]:
gerri = GetMultiDBData(NAIPData(features=["rgb", "ir"]), tile_size=500)

The default online source `{self.base_url}` will be used instead.


# Attention, currently, we are overwriting the files when we enter new locations!

In [24]:
gerri.run(points)

As no dates were specified the most recent data will be fetched.
logged in
The directory for database 'NAIP western europe Azure' will be initialized.
Directory /var/folders/h_/fp7vzm1s46x0fbjfhmjqmlsc0000gn/T/data_barrel/data_barrel/naip_western_europe_azure/source was created.
Directory /var/folders/h_/fp7vzm1s46x0fbjfhmjqmlsc0000gn/T/data_barrel/data_barrel/naip_western_europe_azure/rgb was created.
Directory /var/folders/h_/fp7vzm1s46x0fbjfhmjqmlsc0000gn/T/data_barrel/data_barrel/naip_western_europe_azure/ir was created.
https://naipeuwest.blob.core.windows.net/naip-index/rtree/tile_index.dat
/var/folders/h_/fp7vzm1s46x0fbjfhmjqmlsc0000gn/T/data_barrel/data_barrel/naip_western_europe_azure/index_files/tile_index.dat
https://naipeuwest.blob.core.windows.net/naip-index/rtree/tile_index.idx
/var/folders/h_/fp7vzm1s46x0fbjfhmjqmlsc0000gn/T/data_barrel/data_barrel/naip_western_europe_azure/index_files/tile_index.idx
https://naipeuwest.blob.core.windows.net/naip-index/rtree/tiles.p
/var/

## NAIP

<div class="alert alert-block alert-danger">
    <b>What exactly is this <a href="https://planetarycomputer.microsoft.com/dataset/naip#Blob-Storage-Notebook">Azure blob storage</a>?</b> <br> It seems, as if it does not need authentication, but we would, if we used <a href="https://planetarycomputer.microsoft.com/dataset/naip#Example-Notebook">Planetary Computer STAC API</a> by Microsoft or even Google's <a href="https://developers.google.com/earth-engine/datasets/catalog/USDA_NAIP_DOQQ#description">Earth Engine Data Catalog</a>. Where is the caveat? 
</div>

**Answer:** The explanation was too long to reconstruct for me. But the blob storage is really a path to circumvent the access abrrier in this case.

<div class="alert alert-block alert-danger">
    <b>Are the NAIP tiles actually overlapping?</b> <br> I am unsure, whether a location query could lead to multiple tiles within a given year.
</div>

In [6]:
multidb_wrapper.set_directory("aa")

'aa/data_barrel'

In [20]:
multidb_wrapper.set_locations([1,2,2,3], [2,3,4])

AssertionError: The lists for latitudes ({len(longitudes)}) and lon (4) have unequal length. Please adjust.

---

<div class="alert alert-block alert-danger">
    <b>How can I see if my image was stored properly in 4 layers?</b> <br> When saving as tif and switching the axes, do I loose a layer?
</div>